# Recoinize

[https://github.com/nicknochnack/FaceRecognition](https://github.com/nicknochnack/FaceRecognition/blob/main/Facial%20Verification%20with%20a%20Siamese%20Network%20-%20Final.ipynb)

# Setup

## Pull code from GitHub

In [ ]:
%load_ext autoreload
%autoreload 2

from src.helpers.cmd import git_update, print_gpu_name
from src.helpers.install import install_requirements
# git_update("live-edit", force=False, show_output=False)
# install_requirements()

In [65]:
!git fetch
!git pull

remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 101 (delta 59), reused 62 (delta 47), pack-reused 27
Receiving objects: 100% (101/101), 12.69 KiB | 51.00 KiB/s, done.
Resolving deltas: 100% (72/72), completed with 6 local objects.
From https://github.com/photonsquid/Recoinize
   ae7dbb3..cd8604e  live-edit  -> origin/live-edit
Updating ae7dbb3..cd8604e
Fast-forward
 main.ipynb                                        | 100 +++++++++++++++-------
 src/dataset.py                                    |  34 +++++---
 src/helpers/__pycache__/load_data.cpython-310.pyc | Bin 856 -> 866 bytes
 src/helpers/load_data.py                          |  16 ++--
 4 files changed, 101 insertions(+), 49 deletions(-)


In [7]:
print_gpu_name()

'Quadro P5000\n'

## Import dependencies

In [47]:
testing = False

In [66]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

# Import tensorflow dependencies - Functional API
if not testing:
    import tensorflow as tf
    from tensorflow import keras
    from keras.layers import Dense, Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
    from keras.models import Sequential, load_model, Model
    from datasets import load_dataset

    from src.models import make_embedding, make_siamese_model
    from src.helpers.setup import set_gpus_growth
    from src.dataset import create_pairs
    from src.preprocess import preprocess

    set_gpus_growth()
else:
    print("Testing mode, skipping tensorflow imports")
    from src.helpers.load_data import load_data
    from src.dataset import create_pairs


# Get and preprocess data

In [73]:
# Load the dataset from HuggingFace
if not testing:
    dataset = load_dataset('photonsquid/coins-euro')
else:
    # load data from local files
    dataset = load_data("data/tests")

Using custom data configuration photonsquid--coins-euro-b75261052d3e19e0
Reusing dataset parquet (/root/.cache/huggingface/datasets/photonsquid___parquet/photonsquid--coins-euro-b75261052d3e19e0/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/3 [00:00<?, ?it/s]

In [74]:
# Split the dataset into train and test
train_dataset = dataset['train']
test_dataset = dataset['test']

In [75]:
train_dataset[0]['labels']

{'value': '10cents', 'country': 'ad', 'edition': 'standard', 'variant': '0'}

In [76]:
# create the pairs and labels for the training dataset
train_dataset = create_pairs(train_dataset)

# convert it to a tensorflow dataset
# train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset)

10cents
1cent
1euro
20cents
2cents
2euro
50cents
5cents


In [80]:
# show first pair in subplot with plt

byte_img = train_dataset[0][0]

# img = cv2.imdecode(np.frombuffer(byte_img, np.uint8), cv2.IMREAD_COLOR)
# plt.imshow(img)
# plt.show()


TypeError: a bytes-like object is required, not 'dict'

In [ ]:
# prepare the dataset for training
train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# prepare the dataset for testing
test_dataset = test_dataset.shuffle(1000)
test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)